**Setup training and testing sets**

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import KFold

df = pd.read_csv('sayad_dataset.csv')
target_names = ["no_fire", "fire"]  # 0 for no_fire and 1 for fire

X = df[['NDVI', 'LST', 'BURNED_AREA']].to_numpy()
X = preprocessing.normalize(X)  # Normalize between 0 and 1 to match NDVI

y = df['CLASS'].to_numpy()
def class_identifier(n):        # Convert the no_fire or fire labels to 0 or 1
  if n == 'fire':
    return 1
  else:
    return 0
y = np.array(list(map(class_identifier, y)))

train_index = 731
train_set = X[:train_index]     # The 2D array of training values
train_labels = y[:train_index]  # The 1D array of target labels for the training set

test_index = 1031
test_set = X[train_index:test_index]      # The 2D array of testing values
test_labels = y[train_index:test_index]   # The 1D array of target labels for the testing set

**Build and train the model**

In [ ]:
kfold = KFold(n_splits=10, shuffle=True)

fold_num = 1
for train, test in kfold.split(X, y): # Start kfold cross validation

  model = tf.keras.Sequential([
      # The input shape might not be (731, 3) after we split the dataset into training/testing
      tf.keras.layers.InputLayer(input_shape=train_set.shape[1]),
      tf.keras.layers.Dense(8, activation='sigmoid', use_bias=True), # Hidden layer
      tf.keras.layers.Dense(4, activation='sigmoid', use_bias=True), # Hidden layer
      tf.keras.layers.Dense(2, activation='sigmoid', use_bias=False) # Output layer, two options: 0 or 1
  ])

  # Compile the model with a loss function, optimizer and metrics for checking accuracy
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  print(f"------------- Training for fold {fold_num} -------------")
  # Train the model
  model.fit(X[train], y[train], epochs=10)

  # Evaluate model accuracy using testing set
  print('\n')
  test_loss, test_acc = model.evaluate(X[test],  y[test], verbose=0)
  print(f'\nTest accuracy for fold {fold_num}: {test_acc}\n')

  fold_num += 1

**Make predictions and check their label**

In [ ]:
# Add a softmax layer to the model to convert output into probablities (easier to interpret)
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()]) 

# Predictions is an array of 2 values, each represents the models confidence that the values correspond no_fire or fire
predictions = probability_model.predict(test_set)

for i in range(len(predictions)):
  print(f"Prediction: " + target_names[ np.argmax(predictions[i]) ]) # Check the prediction of the model for the first item in the test set
  print(f"Actual: " + target_names[ int(test_labels[i]) ] + "\n")